In [1]:
!pip install requests
!pip install psycopg2
!pip install pandas

In [2]:
!pip install scikit-learn

In [3]:
import requests
import psycopg2
from psycopg2 import sql
import json
import math
import itertools

url_api = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmueblesPorUsuario/get_filtro/'
url_api_inmuebles = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmuebles/'
url_api_login = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/login/'
url_api_preferenciasPorUsuario = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/interesesPorUsuario/'

login_data = {
    'username': 'diego3026',
    'password': '3174748557d'
}

In [4]:
def normalizacion(valor,valor_maximo,valor_minimo):
    valor_normalizado = (valor - valor_minimo) / (valor_maximo - valor_minimo) 
    return valor_normalizado

In [5]:
class InmuebleUsuarioNormalizado():

    def __init__(self,inmueble,usuario,favorito,calificacion,clics,pesos):
        self.inmueble = inmueble
        self.usuario = usuario
        self.favorito = favorito
        self.calificacion = calificacion
        self.clics = clics
        self.pesos = pesos

    def get_clasificacion(self):
        clasificacion = self.pesos[0] * self.favorito + self.pesos[1] * self.calificacion + self.pesos[2] * self.clics
        return clasificacion


In [6]:
def get_datos_api():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        print(f'Error en la petición login: {response_login.status_code}')


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [7]:
def get_datos_preferenciasPorUsuario():
    response_login = requests.post(url=url_api_login, data=login_data)
    
    if response_login.status_code == 200:
        data = response_login.json()
    else:
        print(f'Error en la petición login: {response_login.status_code}')
        return None
    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response_preferenciasPorUsuario = requests.get(url_api_preferenciasPorUsuario, headers=headers)
    
    if response_preferenciasPorUsuario.status_code == 200:
        data_preferenciasPorUsuario = response_preferenciasPorUsuario.json()
        print(data_preferenciasPorUsuario)
        return data_preferenciasPorUsuario
    else :
        print(f'Error en la peticion de preferenciaPorUsuario: {response_preferenciasPorUsuario}')
        return None
        

In [8]:
preferencias = get_datos_preferenciasPorUsuario()

[{'id': 2, 'interes': 'wifi', 'usuario': 'diego3026'}, {'id': 3, 'interes': 'piscina', 'usuario': 'savarita'}, {'id': 4, 'interes': 'gimnasio', 'usuario': 'savarita'}, {'id': 6, 'interes': 'gimnasio', 'usuario': 'diego3026'}, {'id': 7, 'interes': 'seguridad-247', 'usuario': 'camilo'}, {'id': 9, 'interes': 'piscina', 'usuario': 'diego3026'}, {'id': 12, 'interes': 'terraza', 'usuario': 'diego3026'}, {'id': 14, 'interes': 'seguridad-247', 'usuario': 'dieguito1234'}, {'id': 15, 'interes': 'cocina-integral', 'usuario': 'dieguito1234'}, {'id': 16, 'interes': 'cerca-de-transporte-publico', 'usuario': 'dieguito1234'}, {'id': 17, 'interes': 'piscina', 'usuario': 'calethxd'}, {'id': 18, 'interes': 'jardin', 'usuario': 'calethxd'}, {'id': 19, 'interes': 'mascotas-permitidas', 'usuario': 'calethxd'}, {'id': 20, 'interes': 'piscina', 'usuario': 'juan'}, {'id': 21, 'interes': 'gimnasio', 'usuario': 'juan'}, {'id': 22, 'interes': 'mascotas-permitidas', 'usuario': 'juan'}, {'id': 23, 'interes': 'segur

In [9]:
intereses_por_usuario = {}
for preferencia in preferencias:
    usuario = preferencia['usuario']
    interes = preferencia['interes']
    if usuario not in intereses_por_usuario:
        intereses_por_usuario[usuario] = []
    intereses_por_usuario[usuario].append(interes) 
print(intereses_por_usuario)

{'diego3026': ['wifi', 'gimnasio', 'piscina', 'terraza'], 'savarita': ['piscina', 'gimnasio'], 'camilo': ['seguridad-247'], 'dieguito1234': ['seguridad-247', 'cocina-integral', 'cerca-de-transporte-publico'], 'calethxd': ['piscina', 'jardin', 'mascotas-permitidas'], 'juan': ['piscina', 'gimnasio', 'mascotas-permitidas'], 'R3aper001_': ['seguridad-247', 'piscina', 'mascotas-permitidas', 'cerca-de-transporte-publico', 'vista-al-mar'], 'juan_perez': ['piscina', 'terraza', 'gimnasio', 'jardin', 'cerca-de-transporte-publico'], 'alfredito': ['gimnasio', 'cocina-integral', 'terraza', 'jardin', 'seguridad-247', 'estacionamiento', 'cerca-de-transporte-publico'], 'diego3': ['gimnasio', 'piscina', 'cerca-de-transporte-publico']}


In [10]:
def consultar_base_de_datos(host, database, user, password, consulta, parametros=None):
    """
    Conecta a la base de datos PostgreSQL y ejecuta una consulta.

    :param host: Dirección del servidor PostgreSQL.
    :param database: Nombre de la base de datos.
    :param user: Nombre de usuario para la conexión.
    :param password: Contraseña del usuario.
    :param consulta: Consulta SQL a ejecutar.
    :param parametros: Parámetros para la consulta SQL (opcional).
    :return: Resultados de la consulta.
    """
    try:
        # Establecer la conexión a la base de datos
        conexion = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        cursor = conexion.cursor()

        # Ejecutar la consulta
        cursor.execute(consulta)

        nombres_columnas = [desc[0] for desc in cursor.description]

        # Obtener los resultados de la consulta
        resultados = cursor.fetchall()

        # Cerrar el cursor y la conexión
        cursor.close()
        conexion.close()

        return nombres_columnas, resultados
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error al acceder a la base de datos: {error}')
        return None

In [11]:
host = 'inmuebles.postgres.database.azure.com'
database = 'postgres'
user = 'gidsyc'
password = 'Semillero2024'
consulta = 'SELECT id FROM inmobiliaria_inmueble'
consulta_user = 'SELECT id, username FROM inmobiliaria_usuario'

columnas, resultados = consultar_base_de_datos(host, database, user, password, consulta)

columnas_user, resultados_user = consultar_base_de_datos(host, database, user, password, consulta_user)

datosLimpios = {}

# if resultados:
#     print(columnas)
#     for fila in resultados:
#         print(fila[0])
# else:
#     print('No se obtuvieron resultados.')


# print("Usuarios")
# if resultados_user:
#     print(columnas_user)
#     for fila in resultados_user:
#         print(fila[1])
# else:
#     print('No se obtuvieron resultados.')


for item in resultados_user:
    usuario = item[1]
    if usuario not in datosLimpios:
        datosLimpios[usuario] = {}
        for inmueble in resultados:
            id_inmueble = inmueble[0]
            datosLimpios[usuario][id_inmueble] = None
    else:
        print(f"El inmueble '{id_inmueble}' ya existe")


In [12]:
data = get_datos_api()

#favorito, calificacion, clics
pesos = [5, 4, 1]

calificaciones = []
clics = []
favoritos = []
for item in data:
    if item['calificacion'] != None:
        calificaciones.append(item['calificacion'])

    if item['numeroDeClicks'] != None:
        clics.append(item['numeroDeClicks'])

    if item['favorito'] != None:
        favoritos.append(item['favorito'])

promedio_calificaciones = 1
promedio_clics = 0
promedio_favoritos = 0
max_clics = 0

if len(calificaciones) > 0:
    promedio_calificaciones = sum(calificaciones) / len(calificaciones)

if len(clics) > 0:
    max_clics = max(clics)
    promedio_clics = sum(clics) / len(clics)

if len(favoritos) > 0:
    promedio_favoritos = sum(favoritos) / len(favoritos)

for value in data:
    id_inmueble = value['inmueble'] if value['inmueble']!=None else None
    usuario = value['usuario'] if value['usuario']!=None else None
    calificacion = normalizacion(value['calificacion'],5,1) if value['calificacion']!=None else promedio_calificaciones
    favorito = int(value['favorito']) if value['favorito']!=None else promedio_favoritos

    if max_clics > 0 and value['numeroDeClicks']!=None:
        clics = normalizacion(value['numeroDeClicks'],valor_maximo=max_clics,valor_minimo=0)
    else:
        clics = promedio_clics


    if id_inmueble is not None and usuario is not None:        
        inmueble_por_usuario = InmuebleUsuarioNormalizado(inmueble=id_inmueble,
                                                        usuario=usuario,
                                                        calificacion=calificacion,
                                                        favorito=favorito,
                                                        clics=clics,
                                                        pesos=pesos)
        datosLimpios[inmueble_por_usuario.usuario][inmueble_por_usuario.inmueble] = inmueble_por_usuario.get_clasificacion()

print(datosLimpios)


# Nombre del archivo donde se guardarán los datos
nombre_archivo = 'datosLimpios.json'

# Guardar el diccionario en un archivo JSON
with open(nombre_archivo, 'w') as archivo_json:
    json.dump(datosLimpios, archivo_json, indent=4)


{'mafe': {56: None, 57: None, 58: None, 55: None, 59: None, 111: None, 112: None, 113: None, 114: None, 115: None, 148: None, 149: None, 51: None, 43: None, 44: None, 45: None, 46: None, 47: None, 48: None, 49: None, 50: None, 52: None, 53: None, 54: None, 70: None, 60: None, 61: None, 62: None, 63: None, 64: None, 65: None, 66: None, 67: None, 68: None, 69: None, 71: None, 72: None, 73: None, 74: None, 75: None, 76: None, 77: None, 78: None, 79: None, 99: None, 80: None, 81: None, 82: None, 83: None, 84: None, 85: None, 86: None, 87: None, 88: None, 100: None, 89: None, 90: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 97: None, 98: None, 101: None, 102: None, 103: None, 104: None, 105: None, 106: None, 107: None, 108: None, 109: None, 110: None, 116: None, 117: None, 118: None, 119: None, 120: None, 121: None, 122: None, 123: None, 124: None, 125: None, 126: None, 127: None, 128: None, 129: None, 130: None, 131: None, 132: None, 133: None, 134: None, 135: None, 13

In [13]:
import pandas as pd

ratings_df = pd.DataFrame(datosLimpios).T

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(ratings_df.fillna(0))
cosine_sim_df = pd.DataFrame(cosine_sim, index=ratings_df.index, columns=ratings_df.index)
print("Matriz de similitud del coseno:\n", cosine_sim_df)

Matriz de similitud del coseno:
               mafe  diego3025  savarita  albeiro  camiloproxd  danielr  \
mafe           0.0        0.0       0.0      0.0          0.0      0.0   
diego3025      0.0        0.0       0.0      0.0          0.0      0.0   
savarita       0.0        0.0       0.0      0.0          0.0      0.0   
albeiro        0.0        0.0       0.0      0.0          0.0      0.0   
camiloproxd    0.0        0.0       0.0      0.0          0.0      0.0   
danielr        0.0        0.0       0.0      0.0          0.0      0.0   
calethproxd    0.0        0.0       0.0      0.0          0.0      0.0   
eddie          0.0        0.0       0.0      0.0          0.0      0.0   
calethxd       0.0        0.0       0.0      0.0          0.0      0.0   
wil2021        0.0        0.0       0.0      0.0          0.0      0.0   
Mateo          0.0        0.0       0.0      0.0          0.0      0.0   
carlos14       0.0        0.0       0.0      0.0          0.0      0.0   
juan 

C:\Users\Jonathan Vizcaino\AppData\Local\Temp\ipykernel_9552\1402813186.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cosine_sim = cosine_similarity(ratings_df.fillna(0))


In [15]:
def predict_ratings(user, ratings_df, sim_df):
    user_ratings = ratings_df.loc[user]  # Valoraciones del usuario objetivo
    sim_scores = sim_df[user]  # Similitudes del usuario con otros usuarios
    
    # Excluir el propio usuario de la suma ponderada
    sim_scores = sim_scores.drop(user)
    ratings_df = ratings_df.drop(user)
    
    # Suma ponderada de las valoraciones de los vecinos
    weighted_sum = (ratings_df.T * sim_scores).sum(axis=1)
    sim_sum = sim_scores.sum()
    
    predicted_ratings = weighted_sum / sim_sum
    predicted_ratings = predicted_ratings[user_ratings.isna()]
    predicted_ratings = predicted_ratings[predicted_ratings > 0]

    return predicted_ratings.sort_values(ascending=False)


predicted_ratings = predict_ratings("camilo", ratings_df, cosine_sim_df)
print("Predicciones de valoraciones para 'camilo':\n", predicted_ratings)

recommendations = predicted_ratings.index.tolist()
print(f"inmuebles recomendados para 'camilo': {recommendations}")


Predicciones de valoraciones para 'camilo':
 111    8.421565
113    5.255467
59     5.204194
dtype: object
inmuebles recomendados para 'camilo': [111, 113, 59]


In [16]:
def get_datos_api_inmuebles():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        return f'Error en la petición login: {response_login.status_code}'


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api_inmuebles, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [17]:
inmuebles = get_datos_api_inmuebles()

In [18]:
import re
def buscar_palabra_clave(texto, palabra_clave):
    patron = r'\b' + re.escape(palabra_clave) + r'\b'
    encontrado = re.search(patron, texto) is not None
    return texto, encontrado

In [19]:
!pip install nltk

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')

def extraer_todas_las_palabras(texto):
    # Tokenizar el texto
    texto = texto.replace('-',' ')
    palabras = word_tokenize(texto.lower())
    
    # Filtrar signos de puntuación
    palabras = [palabra for palabra in palabras if palabra not in string.punctuation]

    # Filtrar palabras vacías (stopwords)
    palabras = [palabra for palabra in palabras if palabra not in stopwords.words('spanish')]

    return palabras


[nltk_data] Downloading package punkt to C:\Users\Jonathan
[nltk_data]     Vizcaino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jonathan
[nltk_data]     Vizcaino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
inmueble_caracteristicas = []

for item in inmuebles:
    caracteristicas = item['caracteristicas']
    if (len(caracteristicas) > 0):
        for caracteristica in caracteristicas:
            nombre = caracteristica['nombre'].lower()
            dato = {'id_inmueble': item['id'], 'nombre_caracteristica': nombre}
            inmueble_caracteristicas.append(dato)


In [22]:
inmuebles_con_caracteristicas = {}
for key, group in itertools.groupby(sorted(inmueble_caracteristicas, key=lambda x: x['id_inmueble']), lambda x: x['id_inmueble']):
    inmuebles_con_caracteristicas[key] = [item['nombre_caracteristica'] for item in group]
print(inmuebles_con_caracteristicas)


{43: ['zona-de-lavanderia', 'ascensor', 'circuito-cerrado-de-tv', 'gimnasio', 'piscina', 'salon-comunal', 'vista-panoramica', 'hall-de-alcobas', 'tipo-de-comedor-sala-comedor', 'cuarto-de-servicio', 'zonas-verdes', 'comedor-auxiliar', 'citofonos', 'parqueadero-cubierto', 'instalacion-de-gas-natural', 'acceso-para-discapacitados', 'conjunto-cerrado', 'vigilancia-24hrs', 'zona-para-ninos', 'estudio-o-biblioteca', 'jacuzzi', 'sauna-turco', 'area-terraza-balcon-60-m2', 'numero-de-ascensores-4', 'terraza-balcon-terraza', 'puerta-seguridad', 'closets-4', 'piso-25', 'calentador-caldera', 'tipo-de-cocina-en-u', 'cancha-s-de-squash'], 45: ['cocina-integral', 'jardin', 'bano-auxiliar', 'cuarto-de-servicio', 'zonas-verdes', 'patio', 'tipo-de-parqueadero-propio', 'estudio-o-biblioteca', 'caracteristicas-del-parqueadero-cubierto', 'banos-de-servicio', 'deposito-o-cuarto-util', 'piso-en-baldosa', 'piso-en-porcelanato', 'piso-2'], 46: ['ascensor', 'zonas-verdes', 'terraza-balcon-balcon', 'calentador-

In [39]:
coincidencias_todos_usuarios = {}
for usuario, intereses in intereses_por_usuario.items():
    coincidencias_usuario = []
    for inmueble, caracteristicas in inmuebles_con_caracteristicas.items():
        coincidencias_usuario_inmueble = [interes for interes in intereses if interes in caracteristicas]
        if coincidencias_usuario_inmueble:
            coincidencias_usuario.append((inmueble, coincidencias_usuario_inmueble))
    coincidencias_todos_usuarios[usuario] = coincidencias_usuario

for usuario, coincidencias in coincidencias_todos_usuarios.items():
    print(f"Coincidencias para {usuario}:")
    for coincidencia in coincidencias:
        print(f"Inmueble {coincidencia[0]}: {coincidencia[1]}")
    print()

Coincidencias para diego3026:
Inmueble 43: ['gimnasio', 'piscina']
Inmueble 48: ['gimnasio', 'piscina']
Inmueble 49: ['piscina']
Inmueble 54: ['gimnasio', 'piscina']
Inmueble 58: ['gimnasio']
Inmueble 59: ['gimnasio', 'piscina']
Inmueble 60: ['piscina']
Inmueble 61: ['gimnasio', 'piscina']
Inmueble 62: ['gimnasio', 'piscina']
Inmueble 65: ['piscina']
Inmueble 66: ['gimnasio', 'piscina']
Inmueble 68: ['gimnasio', 'piscina']
Inmueble 69: ['gimnasio', 'piscina']
Inmueble 72: ['gimnasio', 'piscina']
Inmueble 86: ['piscina']
Inmueble 87: ['gimnasio', 'piscina']
Inmueble 91: ['gimnasio', 'piscina']
Inmueble 94: ['gimnasio', 'piscina']
Inmueble 95: ['gimnasio']
Inmueble 96: ['gimnasio', 'piscina']
Inmueble 100: ['piscina']
Inmueble 102: ['gimnasio', 'piscina']
Inmueble 107: ['gimnasio', 'piscina']
Inmueble 113: ['piscina']
Inmueble 118: ['gimnasio', 'piscina']
Inmueble 119: ['gimnasio']
Inmueble 120: ['gimnasio', 'piscina']
Inmueble 131: ['gimnasio', 'piscina']
Inmueble 134: ['piscina']
Inmue

In [26]:
def valoresCoincidenciaPorUsuario(maxPuntaje: int):
    valoresPorUsuarios = []
    for usuario, intereses in intereses_por_usuario.items():
        valorPorUsuario = {"usuario": usuario, "valorCoincidencia": (maxPuntaje/ len(intereses))}
        valoresPorUsuarios.append(valorPorUsuario)
    return valoresPorUsuarios   
    
vCoincidenciasPorUsuario = valoresCoincidenciaPorUsuario(10)    
print(vCoincidenciasPorUsuario)

[{'usuario': 'diego3026', 'valorCoincidencia': 2.5}, {'usuario': 'savarita', 'valorCoincidencia': 5.0}, {'usuario': 'camilo', 'valorCoincidencia': 10.0}, {'usuario': 'dieguito1234', 'valorCoincidencia': 3.3333333333333335}, {'usuario': 'calethxd', 'valorCoincidencia': 3.3333333333333335}, {'usuario': 'juan', 'valorCoincidencia': 3.3333333333333335}, {'usuario': 'R3aper001_', 'valorCoincidencia': 2.0}, {'usuario': 'juan_perez', 'valorCoincidencia': 2.0}, {'usuario': 'alfredito', 'valorCoincidencia': 1.4285714285714286}, {'usuario': 'diego3', 'valorCoincidencia': 3.3333333333333335}]


In [ ]:
def buscarVCUsuario(nUsuario: str):
    usuario = None
    for i in range(len(vCoincidenciasPorUsuario)):
        usuNombre = vCoincidenciasPorUsuario[i]['usuario']
        if usuNombre == nUsuario:
            usuario = vCoincidenciasPorUsuario[i]
            break
    return usuario

def puntajesUsuarios():
    puntajePorUsuarios = []
    for usuario, coincidencias in coincidencias_todos_usuarios.items():
        valorCoincidenciaUsuario = buscarVCUsuario(usuario) 
        puntajesPorInmueble = []
        if valorCoincidenciaUsuario is not None:
            for coincidencia in coincidencias:
                inmuebleP = {"idimueble": coincidencia[0], "puntaje": (len(coincidencia[1]) * valorCoincidenciaUsuario["valorCoincidencia"])}
                puntajesPorInmueble.append(inmuebleP)
        puntajesInmueblesUsuario = {"usuario": usuario, "puntajePorInmueble": puntajesPorInmueble}
        puntajePorUsuarios.append(puntajesInmueblesUsuario)
    return puntajePorUsuarios

puntajesPorUsuarios = puntajesUsuarios()
print(puntajesPorUsuarios)

: 